In [29]:
# !nvidia-smi

In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import tensorflow as tf
import numpy as np, os
from glob import glob
import scipy

IMAGE_SIZE = [224, 224]

train_path = 'D:/breakHis project/datasets/train'
test_path = 'D:/breakHis project/datasets/test'

l = len(os.listdir(train_path))

vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights=None, include_top=False)

for layer in vgg16.layers:
    layer.trainable = True
    
folders = glob('D:/breakHis project/datasets/train/*')

# our layers - you can add more if you want
x = Flatten()(vgg16.output)

prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg16.input, outputs=prediction)

model.compile(
  loss='categorical_crossentropy',
#   optimizer='adam',
  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
  metrics=['accuracy']
)

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
#                                    shear_range = 0.2,
#                                    zoom_range = 0.2,
#                                   horizontal_flip = True
                                   )

test_datagen = ImageDataGenerator(rescale = 1./255)

# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

# fit the model
epochs = 10
# Run the cell. It will take some time to execute
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=epochs,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)


Found 1184 images belonging to 2 classes.
Found 509 images belonging to 2 classes.
Epoch 1/10
37/37 [==============================] - ETA: 0s - loss: 0.7066 - accuracy: 0.6689 

In [ ]:
import matplotlib.pyplot as plt
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
y_pred = model.predict(test_set)
import numpy as np
y_pred = np.argmax(y_pred, axis=1)
# y_pred

In [ ]:
from sklearn.metrics import confusion_matrix
y_true = test_set.labels
cm=confusion_matrix(y_true,y_pred)
cm

In [ ]:
import seaborn as sns
plt.figure(figsize=(10,6))
sns.heatmap(cm,annot=True)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')

In [ ]:
acc = r.history['accuracy']
val_acc = r.history['val_accuracy']

loss = r.history['loss']
val_loss = r.history['val_loss']


In [ ]:
print(classification_report(y_true, y_pred))

In [ ]:
from sklearn.metrics import accuracy_score, f1_score,matthews_corrcoef
score=accuracy_score(y_true,y_pred)
f1 = f1_score(y_true,y_pred)
mcc = matthews_corrcoef(y_true,y_pred)
print(f"accuracy {score} f1 {f1} mcc {mcc}")